<a href="https://colab.research.google.com/github/Manoj048/Car-price-Regression/blob/main/02_WordEmbeddings_Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import gensim
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize
from gensim.models import Word2Vec
import os

In [3]:
from sklearn.datasets import fetch_20newsgroups

categories = [
    'talk.religion.misc',
    'comp.graphics',
    'sci.space',
]

print("Loading 20 newsgroups dataset for categories:")
print(categories)

Loading 20 newsgroups dataset for categories:
['talk.religion.misc', 'comp.graphics', 'sci.space']


In [4]:
df = fetch_20newsgroups(subset='all', categories=categories, 
                             shuffle=False, remove=('headers', 'footers', 'quotes'))
labels = df.target
true_k = len(np.unique(labels)) ## This should be 3 in this example
print(true_k)

3


In [5]:
df.data[1]

'In regards to fractal commpression, I have seen 2 fractal compressed "movies".\nThey were both fairly impressive.  The first one was a 64 gray scale "movie" of\nCasablanca, it was 1.3MB and had 11 minutes of 13 fps video.  It was a little\ngrainy but not bad at all.  The second one I saw was only 3 minutes but it\nhad 8 bit color with 10fps and measured in at 1.2MB.\n\nI consider the fractal movies a practical thing to explore.  But unlike many \nother formats out there, you do end up losing resolution.  I don\'t know what\nkind of software/hardware was used for creating the "movies" I saw but the guy\nthat showed them to me said it took 5-15 minutes per frame to generate.  But as\nI said above playback was 10 or more frames per second.  And how else could you\nput 11 minutes on one floppy disk?'

In [6]:
len(df.data)

2588

In [7]:
data=[]
for text in df.data:
  data.append(text)

In [8]:
data[0]

'\n\nI think I can. Largely as a result of efforts by people reading this group\nwriting letters and making phone calls the following has happened:\n\n1. NASA reprogrammed funds to keep NASP alive in 1991.\n2. Efforts to kill DC-X and the SSRT progam where twice twarted\n   (Feb. and June of last year).\n3. Gouldin kept his job in spite of heavy lobbying against him.\n\nThis may not be what Mark was thinking of but it shows that the\nreaders of sci.space DO have power and influence.\n\n  Allen\n'

In [9]:
new=[]
# data is a list formed by all entries of df.data
for i in data:
  i=re.sub('[\s+\d+:\.\)\( ]',' ',i) #'\.' spaces, numbers, colon, paranthesis, full stop rmoval
  i=re.sub(r'\S*@\S*\s?','',i)  # Email removal
  new.append(i)

new[0:3]
  

['  I think I can  Largely as a result of efforts by people reading this group writing letters and making phone calls the following has happened      NASA reprogrammed funds to keep NASP alive in          Efforts to kill DC-X and the SSRT progam where twice twarted     Feb  and June of last year      Gouldin kept his job in spite of heavy lobbying against him   This may not be what Mark was thinking of but it shows that the readers of sci space DO have power and influence     Allen ',
 'In regards to fractal commpression, I have seen   fractal compressed "movies"  They were both fairly impressive   The first one was a    gray scale "movie" of Casablanca, it was    MB and had    minutes of    fps video   It was a little grainy but not bad at all   The second one I saw was only   minutes but it had   bit color with   fps and measured in at    MB   I consider the fractal movies a practical thing to explore   But unlike many  other formats out there, you do end up losing resolution   I don

In [10]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [11]:
# new is a list that contains all the data entries from dataframe after preprocessing
tokens=[]
for text in new:
  text= re.sub(r'\s+',' ',text.lower())
  tokens.append(word_tokenize(text))
  
 

In [12]:
tokens[0:2]
print(len(tokens))
print(len(tokens[0]))
print(len(tokens[1]))

2588
85
154


For more details on the arguements
https://radimrehurek.com/gensim/models/word2vec.html#:~:text=hs%20(%7B0%2C%201%7D,usually%20between%205%2D20).

In [16]:
# corpus, 
# size = vector dimension of the word representation,
#size: The number of dimensions of the embeddings and the default is 100.

#window: The maximum distance between a target word and words around the target word. The default window is 5.

#min_count: The minimum count of words to consider when training the model; 
#words with occurrence less than this count will be ignored. The default for min_count is 5.
# sg: The training algorithm, either CBOW(0) or skip gram(1). The default training algorithm is CBOW.
#  hs ({0, 1}, optional) – If 1, hierarchical softmax will be used for model training. 
# If 0, and negative is non-zero, negative sampling will be used
Gensim_model = Word2Vec(tokens,vector_size=50,window=5,sg=1,hs=0, epochs=10 )

In [17]:
w1='software'
Gensim_model.wv.most_similar(positive=w1,topn=4)

[('documentation', 0.7871972918510437),
 ('portable', 0.7226742506027222),
 ('imdisp', 0.6992469429969788),
 ('grass', 0.6961449384689331)]

In [18]:
w1='science'
Gensim_model.wv.most_similar(positive=w1,topn=4)

[('bachelor', 0.6907857060432434),
 ('sciences', 0.6907344460487366),
 ('electro-magnetic', 0.6856871843338013),
 ('department', 0.6664952039718628)]

In [22]:
w1='technology'
Gensim_model.wv.most_similar(negative=w1,topn=4)

[('-q', 0.11141394823789597),
 ('hand', 0.09498467296361923),
 ('ps', 0.07368414849042892),
 ('followed', 0.07055161893367767)]

In [23]:
w1='technology'
Gensim_model.wv.most_similar(positive=w1,topn=4)

[('aerospace', 0.8298055529594421),
 ('marketing', 0.7677119374275208),
 ('policy', 0.766974687576294),
 ('demonstrator', 0.7635629773139954)]

In [20]:
Gensim_model.wv.get_vector('science')

array([-0.15884385,  0.11386719,  1.0407876 ,  0.77489406, -0.16279471,
        0.5885213 ,  0.642327  , -0.15874533, -0.3307163 , -0.46232626,
       -0.6740171 , -0.22653297, -0.2508748 ,  0.44945198, -0.3649284 ,
        0.32042414,  1.583339  ,  0.03832008, -0.81387514, -0.2818489 ,
       -0.05262995,  0.3514307 ,  0.17602348,  0.24945197,  0.06506569,
        0.23610637,  0.40906298, -0.64573395, -0.1414611 ,  0.084113  ,
        0.08860959,  0.44852197,  0.17711759,  0.22544359,  0.18289371,
        0.18594237, -0.11017715, -0.1875838 , -0.7586856 , -0.47800595,
        0.08023006,  0.12059887,  0.23713261,  0.18070649,  0.7567296 ,
       -0.11049952, -0.46832424, -0.3864114 ,  0.14369407,  0.1065428 ],
      dtype=float32)

In [21]:
Gensim_model.wv.most_similar('Gastroenteritis')#no similar words

KeyError: ignored

Conceptually Word2Vec and fastText have the same goal: to learn vector representations of words. But unlike Word2Vec, which under the hood uses words to predict words, fastText operates at a more granular level with character n-grams. Where words are represented by the sum of the character n-gram vectors.

https://radimrehurek.com/gensim/models/fasttext.html?highlight=fasttext#module-gensim.models.fasttext

In [25]:
from gensim.models import FastText
model_fast = FastText(tokens, vector_size=100, window=5, min_count=5, workers=4,sg=1) # religion, space and graphics

In [26]:
model_fast.wv.most_similar("Gastroenteritis")

[('amateur', 0.9448818564414978),
 ('industrial', 0.9404093623161316),
 ('strategic', 0.9354268908500671),
 ('pacastro', 0.9311636090278625),
 ('chemistry', 0.9300380945205688),
 ('auxiliary', 0.9260054230690002),
 ('astral', 0.9242093563079834),
 ('assessment', 0.9237842559814453),
 ('category', 0.9230931401252747),
 ('extraterrestrial', 0.9212397336959839)]